<a href="https://colab.research.google.com/github/rodrigoms95/herramientas-climatico-22-1/blob/main/code/Proyecto/Jupyter/Data_proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Procesa el dataframe para el machine learning.

import pandas as pd

In [2]:
path_drive = "/content/drive/MyDrive/Colab/" 
path_data = path_drive + "data/data_"
path_socio = path_drive + "Socioeconomicos/"
csv = ".csv"
socio = [ "Electrico_residencial_2010_2017.csv",
    "Datos demográficos.csv", "GLP_2010_2016.csv" ]

data = pd.read_csv(path_data + str(0) + csv)

In [3]:
# Agrega las variables procesadas para cada municipio.
for i in range(1, 10):
    data_i = pd.read_csv(path_data + str(i) + csv)
    data.iloc[:, i + 10] = data_i.iloc[:,i + 3]

In [4]:
data_1 = data.copy()

data.rename(columns = {"Año": "Year"}, inplace = True)

# Agrega el precio de gas LP.
df_socio = pd.read_csv(path_socio + socio[2])
for i in data.itertuples():
  data_1.loc[i[0], "$GLP"] = df_socio.loc[
        df_socio["CVE_ENT"] == i.CVE_ENT, str(i.Year)].values[0]

# Agrega la población y el PIB.    
df_socio = pd.read_csv(path_socio + socio[1])
for i in data.itertuples():
    for k in ["Población", "PIB"]:
        data_1.loc[i[0], k ] = df_socio.loc[
            df_socio["CVE_INEGI"] == i.CVE_INEGI,
            k + " " + str(i.Year)].values[0]

# Agrega el consumo y la cantidad de usuarios.
df_socio = pd.read_csv(path_socio + socio[0])
for i in data.itertuples():
    for k in ["Consumo", "Usuarios"]:
        for l in ["1*", "DAC"]:
            data_1.loc[i[0], k + "_" + l ] = df_socio.loc[
                (df_socio["CVE_INEGI"] == i.CVE_INEGI)
                & df_socio["Tarifa"].str.contains(l, regex = True),
                k + " " + str(i.Year)].values[0]

# PIB per cápita y densidad de población.
data_1["PCI"] = data_1["PIB"] / data_1["Población"]
data_1["Densidad_población"] = data_1["Población"] / data_1["Area"]

data = data_1

In [5]:
# Escogemos las variables relevantes.
data_proc = data.drop(["Area", "CVE_ENT", "CVE_MUN",
  "T_max", "T_min", "HDD_mean",
  "HDD_p10",  "CDD_p90", "$luz"], axis = 1)
data_proc = data_proc[list(data_proc.columns[1:]) + ["CVE_INEGI"]]

data_proc["Consumo_1*"] += data_proc["Consumo_DAC"]
data_proc["Usuarios_1*"] += data_proc["Usuarios_DAC"]
data_proc.drop(["Consumo_DAC", "Usuarios_DAC"],
    axis = 1, inplace = True)

data_proc.to_csv(path_data[:-1] + csv, index = False)

# Correlación entre variables.
data_proc.corr()

,Consumo_1*,Usuarios_1*,lon,lat,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
Consumo_1*,1.000000,0.844769,-1.517104e-01,1.966613e-01,0.037609,0.175000,-0.108148,-0.093338,0.318539,0.280889,0.005908,0.807671,0.749397,1.314590e-02,-6.607402e-02
Usuarios_1*,0.844769,1.000000,-1.204962e-01,1.351577e-01,-0.047144,0.095912,-0.112452,-0.082853,0.541620,0.309517,0.017101,0.985200,0.889541,1.865308e-02,-1.208885e-01
lon,-0.151710,-0.120496,1.000000e+00,-6.648263e-01,0.368563,-0.570965,0.487079,0.412035,-0.000243,-0.270591,-0.013115,-0.111694,-0.085190,-4.969590e-18,2.369749e-01
lat,0.196661,0.135158,-6.648263e-01,1.000000e+00,-0.172404,0.901895,-0.477210,-0.406219,-0.029756,0.368796,0.037059,0.121039,0.110049,3.010546e-18,-1.816479e-02
T_mean,0.037609,-0.047144,3.685626e-01,-1.724041e-01,1.000000,0.002264,0.406738,0.400219,-0.174850,-0.001128,0.025122,-0.066071,-0.048065,4.720407e-02,1.749890e-01
CDD_mean,0.175000,0.095912,-5.709655e-01,9.018954e-01,0.002264,1.000000,-0.402018,-0.337884,-0.071880,0.331921,0.036149,0.078861,0.075153,7.924268e-02,-4.882388e-02
Pre,-0.108148,-0.112452,4.870794e-01,-4.772099e-01,0.406738,-0.402018,1.000000,0.921505,-0.071052,-0.257785,-0.093505,-0.105271,-0.096163,-6.036922e-02,1.105634e-01
Pre_Tmean,-0.093338,-0.082853,4.120352e-01,-4.062194e-01,0.400219,-0.337884,0.921505,1.000000,-0.049735,-0.196991,-0.037105,-0.075536,-0.076816,-1.245438e-03,1.161193e-01
Densidad_población,0.318539,0.541620,-2.431198e-04,-2.975554e-02,-0.174850,-0.071880,-0.071052,-0.049735,1.000000,0.229053,0.003081,0.560745,0.585140,5.724501e-03,-7.796424e-02
PCI,0.280889,0.309517,-2.705908e-01,3.687962e-01,-0.001128,0.331921,-0.257785,-0.196991,0.229053,1.000000,0.066342,0.283659,0.473212,4.605540e-02,-1.489635e-01
